# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 19 2022 2:47PM,242246,10,PRF-33625,Bio-PRF,Released
1,Jul 19 2022 2:45PM,242245,10,PRF-33616,Bio-PRF,Released
2,Jul 19 2022 2:45PM,242245,10,PRF-33622,Bio-PRF,Released
3,Jul 19 2022 2:44PM,242244,10,CTF0011327.1,"Citieffe, Inc.",Released
4,Jul 19 2022 2:44PM,242243,10,CTF0011325.1,"Citieffe, Inc.",Released
5,Jul 19 2022 2:37PM,242242,19,TR-RG-07202022-WM6.1,"GCH Granules USA, Inc.",Released
6,Jul 19 2022 2:35PM,242224,18,INNOVA-8295935,Innova Softgel LLC,Released
7,Jul 19 2022 2:31PM,242241,10,Eme-101369,Emerginnova,Released
8,Jul 19 2022 2:31PM,242241,10,Enova-10324,Emerginnova,Released
9,Jul 19 2022 2:27PM,242237,10,0085501605,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,242242,Released,1
35,242243,Released,1
36,242244,Released,1
37,242245,Released,2
38,242246,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242242,NaN,NaN,1.0
242243,NaN,NaN,1.0
242244,NaN,NaN,1.0
242245,NaN,NaN,2.0
242246,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242137,0.0,0.0,1.0
242139,0.0,3.0,63.0
242147,0.0,0.0,1.0
242150,17.0,3.0,4.0
242160,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242137,0,0,1
242139,0,3,63
242147,0,0,1
242150,17,3,4
242160,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242137,0,0,1
1,242139,0,3,63
2,242147,0,0,1
3,242150,17,3,4
4,242160,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242137,,,1
1,242139,,3,63
2,242147,,,1
3,242150,17,3,4
4,242160,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 19 2022 2:47PM,242246,10,Bio-PRF
1,Jul 19 2022 2:45PM,242245,10,Bio-PRF
3,Jul 19 2022 2:44PM,242244,10,"Citieffe, Inc."
4,Jul 19 2022 2:44PM,242243,10,"Citieffe, Inc."
5,Jul 19 2022 2:37PM,242242,19,"GCH Granules USA, Inc."
6,Jul 19 2022 2:35PM,242224,18,Innova Softgel LLC
7,Jul 19 2022 2:31PM,242241,10,Emerginnova
9,Jul 19 2022 2:27PM,242237,10,ISDIN Corporation
76,Jul 19 2022 2:25PM,242240,22,"NBTY Global, Inc."
77,Jul 19 2022 2:03PM,242234,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 19 2022 2:47PM,242246,10,Bio-PRF,,,1
1,Jul 19 2022 2:45PM,242245,10,Bio-PRF,,,2
2,Jul 19 2022 2:44PM,242244,10,"Citieffe, Inc.",,,1
3,Jul 19 2022 2:44PM,242243,10,"Citieffe, Inc.",,,1
4,Jul 19 2022 2:37PM,242242,19,"GCH Granules USA, Inc.",,,1
5,Jul 19 2022 2:35PM,242224,18,Innova Softgel LLC,,,1
6,Jul 19 2022 2:31PM,242241,10,Emerginnova,,,2
7,Jul 19 2022 2:27PM,242237,10,ISDIN Corporation,,,67
8,Jul 19 2022 2:25PM,242240,22,"NBTY Global, Inc.",,1,
9,Jul 19 2022 2:03PM,242234,10,Yusen – 3D Matrix,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 2:47PM,242246,10,Bio-PRF,1,,
1,Jul 19 2022 2:45PM,242245,10,Bio-PRF,2,,
2,Jul 19 2022 2:44PM,242244,10,"Citieffe, Inc.",1,,
3,Jul 19 2022 2:44PM,242243,10,"Citieffe, Inc.",1,,
4,Jul 19 2022 2:37PM,242242,19,"GCH Granules USA, Inc.",1,,
5,Jul 19 2022 2:35PM,242224,18,Innova Softgel LLC,1,,
6,Jul 19 2022 2:31PM,242241,10,Emerginnova,2,,
7,Jul 19 2022 2:27PM,242237,10,ISDIN Corporation,67,,
8,Jul 19 2022 2:25PM,242240,22,"NBTY Global, Inc.",,1,
9,Jul 19 2022 2:03PM,242234,10,Yusen – 3D Matrix,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 2:47PM,242246,10,Bio-PRF,1,,
1,Jul 19 2022 2:45PM,242245,10,Bio-PRF,2,,
2,Jul 19 2022 2:44PM,242244,10,"Citieffe, Inc.",1,,
3,Jul 19 2022 2:44PM,242243,10,"Citieffe, Inc.",1,,
4,Jul 19 2022 2:37PM,242242,19,"GCH Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 2:47PM,242246,10,Bio-PRF,1.0,NaN,NaN
1,Jul 19 2022 2:45PM,242245,10,Bio-PRF,2.0,NaN,NaN
2,Jul 19 2022 2:44PM,242244,10,"Citieffe, Inc.",1.0,NaN,NaN
3,Jul 19 2022 2:44PM,242243,10,"Citieffe, Inc.",1.0,NaN,NaN
4,Jul 19 2022 2:37PM,242242,19,"GCH Granules USA, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 19 2022 2:47PM,242246,10,Bio-PRF,1.0,0.0,0.0
1,Jul 19 2022 2:45PM,242245,10,Bio-PRF,2.0,0.0,0.0
2,Jul 19 2022 2:44PM,242244,10,"Citieffe, Inc.",1.0,0.0,0.0
3,Jul 19 2022 2:44PM,242243,10,"Citieffe, Inc.",1.0,0.0,0.0
4,Jul 19 2022 2:37PM,242242,19,"GCH Granules USA, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3391094,95.0,3.0,0.0
12,242230,2.0,0.0,0.0
15,484346,5.0,3.0,17.0
18,484441,2.0,0.0,0.0
19,968870,29.0,3.0,0.0
20,1211000,99.0,4.0,0.0
21,726444,3.0,0.0,0.0
22,726662,2.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3391094,95.0,3.0,0.0
1,12,242230,2.0,0.0,0.0
2,15,484346,5.0,3.0,17.0
3,18,484441,2.0,0.0,0.0
4,19,968870,29.0,3.0,0.0
5,20,1211000,99.0,4.0,0.0
6,21,726444,3.0,0.0,0.0
7,22,726662,2.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,95.0,3.0,0.0
1,12,2.0,0.0,0.0
2,15,5.0,3.0,17.0
3,18,2.0,0.0,0.0
4,19,29.0,3.0,0.0
5,20,99.0,4.0,0.0
6,21,3.0,0.0,0.0
7,22,2.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,95.0
1,12,Released,2.0
2,15,Released,5.0
3,18,Released,2.0
4,19,Released,29.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0
Executing,3.0,0.0,3.0,0.0,3.0,4.0,0.0,1.0
Released,95.0,2.0,5.0,2.0,29.0,99.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20,21,22
0,Completed,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0
1,Executing,3.0,0.0,3.0,0.0,3.0,4.0,0.0,1.0
2,Released,95.0,2.0,5.0,2.0,29.0,99.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20,21,22
0,Completed,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0
1,Executing,3.0,0.0,3.0,0.0,3.0,4.0,0.0,1.0
2,Released,95.0,2.0,5.0,2.0,29.0,99.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()